# Notebook 03 — Analise Multi-Hazard: Inundacao + Ondas de Calor

**CLIMARISK-OG** | Petrobras | TRL5  
Ativo: REDUC — Refinaria Duque de Caxias, RJ  

Este notebook combina os dois hazards (NB01 + NB02) sobre o **mesmo ativo**
usando uma Exposure multi-hazard e um ImpactFuncSet unificado.

**Versao 1.1** — Correcao: curva JRC via `climada_petals` para consistencia com NB01.

In [ ]:
!pip install climada climada-petals --quiet

In [ ]:
# BLOCO 1 -- Verificacao de Ambiente
import sys
print(f"Python: {sys.version}")

import climada
try:
    ver = climada.__version__
except AttributeError:
    from importlib.metadata import version
    ver = version("climada")
print(f"CLIMADA versao: {ver}")
CLIMADA_OK = True

In [ ]:
# =============================================================================
# BLOCO 2: IMPORTS
# =============================================================================
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# CLIMADA core
from climada.hazard import Hazard, Centroids
from climada.entity import Exposures, ImpactFuncSet, ImpactFunc
from climada.engine import ImpactCalc
from scipy.sparse import csr_matrix

# CLIMADA petals -- funcao de dano JRC para inundacao
from climada_petals.entity.impact_funcs.river_flood import (
    ImpfRiverFlood, flood_imp_func_set
)

matplotlib.rcParams['figure.figsize'] = (12, 8)
matplotlib.rcParams['figure.dpi'] = 100

print("\nOK - Imports carregados com sucesso")

In [ ]:
# =============================================================================
# BLOCO 3: DEFINICAO DO ATIVO (IDENTICO AO NB01/NB02)
# =============================================================================
REDUC_LAT = -22.53
REDUC_LON = -43.28
REDUC_NAME = "REDUC - Refinaria Duque de Caxias"
REDUC_VALUE_USD = 5_000_000_000  # USD 5 bilhoes

BBOX = {
    'lat_min': -23.0, 'lat_max': -22.0,
    'lon_min': -43.8, 'lon_max': -42.8
}

print(f"Ativo: {REDUC_NAME}")
print(f"Coordenadas: ({REDUC_LAT}, {REDUC_LON})")
print(f"Valor de exposicao: USD {REDUC_VALUE_USD:,.0f}")

In [ ]:
# =============================================================================
# BLOCO 4: EXPOSURE MULTI-HAZARD
# =============================================================================
# A Exposure precisa de DUAS colunas impf_: uma para RF e outra para HW.
# O ID da impact function de inundacao (impf_RF) deve corresponder ao ID
# retornado pelo climada_petals para South America.
#
# Carregamos o set JRC completo e extraimos o ID correto.

impf_set_jrc = flood_imp_func_set()

# Listar funcoes disponiveis para conferir IDs
print("Impact Functions JRC disponiveis:")
for haz_type in impf_set_jrc.get_hazard_types():
    for func_id in impf_set_jrc.get_ids().get(haz_type, []):
        func = impf_set_jrc.get_func(haz_type=haz_type, fun_id=func_id)
        if isinstance(func, list):
            func = func[0]
        print(f"  {haz_type} id={func_id}: {func.name}")

# Identificar o ID da funcao South America
RF_SA_ID = None
for func_id in impf_set_jrc.get_ids().get('RF', []):
    func = impf_set_jrc.get_func(haz_type='RF', fun_id=func_id)
    if isinstance(func, list):
        func = func[0]
    if 'South America' in func.name or 'SouthAmerica' in func.name:
        RF_SA_ID = func_id
        break

if RF_SA_ID is None:
    RF_SA_ID = 6  # fallback
    print(f"\nATENCAO: Nao encontrou 'South America' pelo nome. Usando ID={RF_SA_ID}")
else:
    print(f"\nID da funcao JRC South America: {RF_SA_ID}")

# ID da funcao customizada de calor
HW_ID = 1

import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.GeoDataFrame({
    'value': [REDUC_VALUE_USD],
    'latitude': [REDUC_LAT],
    'longitude': [REDUC_LON],
    'impf_RF': [RF_SA_ID],
    'impf_HW': [HW_ID],
    'asset_name': [REDUC_NAME],
}, geometry=[Point(REDUC_LON, REDUC_LAT)], crs="EPSG:4326")

exp = Exposures(gdf)
exp.value_unit = 'USD'
exp.check()

print(f"\nExposure multi-hazard configurada:")
print(f"  impf_RF = {RF_SA_ID} (JRC South America)")
print(f"  impf_HW = {HW_ID} (Custom Heat Wave)")

In [ ]:
# =============================================================================
# BLOCO 5: HAZARD 1 -- INUNDACAO FLUVIAL (IDENTICO AO NB01)
# =============================================================================
print("--- Construindo Hazard de Inundacao ---")

n_lat, n_lon = 20, 20
lats = np.linspace(BBOX['lat_min'], BBOX['lat_max'], n_lat)
lons = np.linspace(BBOX['lon_min'], BBOX['lon_max'], n_lon)
lon_grid, lat_grid = np.meshgrid(lons, lats)

centroids = Centroids(
    lat=lat_grid.flatten(),
    lon=lon_grid.flatten(),
    crs='EPSG:4326'
)
n_centroids = centroids.size

events_rf = [
    {'name': 'flood_rp5',   'rp': 5,   'max_depth': 0.5,  'year': 2020},
    {'name': 'flood_rp10',  'rp': 10,  'max_depth': 1.0,  'year': 2015},
    {'name': 'flood_rp25',  'rp': 25,  'max_depth': 1.8,  'year': 2010},
    {'name': 'flood_rp50',  'rp': 50,  'max_depth': 2.5,  'year': 2000},
    {'name': 'flood_rp100', 'rp': 100, 'max_depth': 3.5,  'year': 1988},
    {'name': 'flood_rp250', 'rp': 250, 'max_depth': 4.5,  'year': 1966},
]
n_events_rf = len(events_rf)

intensity_rf = np.zeros((n_events_rf, n_centroids))
fraction_rf = np.zeros((n_events_rf, n_centroids))

for i, evt in enumerate(events_rf):
    dist = np.sqrt(
        (centroids.lat - REDUC_LAT)**2 +
        (centroids.lon - REDUC_LON)**2
    )
    max_radius = 0.3
    depth = evt['max_depth'] * np.maximum(0, 1 - dist / max_radius)
    np.random.seed(42 + i)
    noise = np.random.normal(1.0, 0.2, n_centroids)
    noise = np.clip(noise, 0.5, 1.5)
    depth = depth * noise
    depth = np.maximum(0, depth)
    intensity_rf[i, :] = depth
    fraction_rf[i, :] = np.where(
        depth > 0.01,
        np.minimum(1.0, depth / evt['max_depth']),
        0.0
    )

haz_flood = Hazard(
    haz_type='RF',
    centroids=centroids,
    event_id=np.arange(1, n_events_rf + 1),
    event_name=[e['name'] for e in events_rf],
    date=np.array([datetime(e['year'], 1, 15).toordinal() for e in events_rf]),
    frequency=np.array([1.0 / e['rp'] for e in events_rf]),
    frequency_unit='1/year',
    intensity=csr_matrix(intensity_rf),
    fraction=csr_matrix(fraction_rf),
    units='m',
)
haz_flood.check()

print(f"  Eventos: {haz_flood.size}")
print(f"  Centroids: {haz_flood.centroids.size}")
print(f"  Intensidade maxima: {haz_flood.intensity.max():.2f} m")
print(f"  OK - Hazard de inundacao construido")

In [ ]:
# =============================================================================
# BLOCO 6: HAZARD 2 -- ONDAS DE CALOR (IDENTICO AO NB02)
# =============================================================================
print("--- Construindo Hazard de Ondas de Calor ---")

HEAT_THRESHOLD_C = 40.0

events_hw = [
    {'name': 'heatwave_rp2',   'rp': 2,   'delta_above': 2.0,  'year': 2023},
    {'name': 'heatwave_rp5',   'rp': 5,   'delta_above': 3.5,  'year': 2020},
    {'name': 'heatwave_rp10',  'rp': 10,  'delta_above': 5.0,  'year': 2016},
    {'name': 'heatwave_rp25',  'rp': 25,  'delta_above': 6.5,  'year': 2010},
    {'name': 'heatwave_rp50',  'rp': 50,  'delta_above': 8.0,  'year': 1998},
    {'name': 'heatwave_rp100', 'rp': 100, 'delta_above': 10.0, 'year': 1984},
]
n_events_hw = len(events_hw)

intensity_hw = np.zeros((n_events_hw, n_centroids))
fraction_hw = np.zeros((n_events_hw, n_centroids))

for i, evt in enumerate(events_hw):
    np.random.seed(100 + i)
    base_temp = evt['delta_above']
    spatial_var = np.random.normal(0, 0.5, n_centroids)
    dist = np.sqrt(
        (centroids.lat - REDUC_LAT)**2 +
        (centroids.lon - REDUC_LON)**2
    )
    urban_heat = np.maximum(0, 0.5 * (1 - dist / 0.3))
    temp_field = base_temp + spatial_var + urban_heat
    temp_field = np.maximum(0, temp_field)
    intensity_hw[i, :] = temp_field
    fraction_hw[i, :] = np.where(temp_field > 0.1, 1.0, 0.0)

haz_heat = Hazard(
    haz_type='HW',
    centroids=centroids,
    event_id=np.arange(1, n_events_hw + 1),
    event_name=[e['name'] for e in events_hw],
    date=np.array([datetime(e['year'], 7, 15).toordinal() for e in events_hw]),
    frequency=np.array([1.0 / e['rp'] for e in events_hw]),
    frequency_unit='1/year',
    intensity=csr_matrix(intensity_hw),
    fraction=csr_matrix(fraction_hw),
    units='deg_C above threshold',
)
haz_heat.check()

print(f"  Eventos: {haz_heat.size}")
print(f"  Intensidade maxima: {haz_heat.intensity.max():.2f} deg_C acima de {HEAT_THRESHOLD_C} deg_C")
print(f"  OK - Hazard de calor construido")

In [ ]:
# =============================================================================
# BLOCO 7: IMPACT FUNCTIONS UNIFICADAS
# =============================================================================
# MUDANCA PRINCIPAL vs v1.0:
# A funcao JRC de inundacao agora vem do climada_petals (mesma do NB01),
# em vez de ser reconstruida manualmente. Isso garante:
#   1. Consistencia NB01 <-> NB03 (mesma curva, mesmo EAI)
#   2. PAA correto (derivado da flooded fraction, nao PAA=1.0)
#   3. Rastreabilidade (fonte: Huizinga et al. 2017 via implementacao oficial)
# =============================================================================
print("--- Configurando Impact Functions ---")

# 7a. Funcao JRC de inundacao via climada_petals
impf_flood = impf_set_jrc.get_func(haz_type='RF', fun_id=RF_SA_ID)
if isinstance(impf_flood, list):
    impf_flood = impf_flood[0]

print(f"  RF: {impf_flood.name}")
print(f"      ID={impf_flood.id}, pontos={len(impf_flood.intensity)}")
print(f"      MDD range: [{impf_flood.mdd.min():.3f}, {impf_flood.mdd.max():.3f}]")
print(f"      PAA range: [{impf_flood.paa.min():.3f}, {impf_flood.paa.max():.3f}]")

# 7b. Funcao customizada de calor (identica ao NB02)
impf_heat = ImpactFunc(
    id=HW_ID,
    haz_type='HW',
    name='Heat Wave -- Industrial Facility (Refinery)',
    intensity_unit='deg_C above threshold',
    intensity=np.array([0.0, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, 15.0]),
    mdd=np.array(       [0.0, 0.02, 0.05, 0.10, 0.25, 0.45, 0.70, 0.95]),
    paa=np.array(       [0.0, 0.10, 0.20, 0.35, 0.55, 0.75, 0.90, 1.00]),
)

print(f"  HW: {impf_heat.name}")
print(f"      ID={impf_heat.id}, pontos={len(impf_heat.intensity)}")

# 7c. ImpactFuncSet unificado
impf_set = ImpactFuncSet([impf_flood, impf_heat])
impf_set.check()
print(f"\n  OK - ImpactFuncSet unificado com {len(impf_set.get_hazard_types())} hazards")

In [ ]:
# =============================================================================
# BLOCO 7b: PLOT DAS IMPACT FUNCTIONS
# =============================================================================
try:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    mdr_rf = impf_flood.mdd * impf_flood.paa
    ax1.plot(impf_flood.intensity, mdr_rf * 100, 'b-o', linewidth=2, markersize=6)
    ax1.fill_between(impf_flood.intensity, 0, mdr_rf * 100, alpha=0.15, color='blue')
    ax1.set_xlabel('Profundidade (m)')
    ax1.set_ylabel('MDR (%)')
    ax1.set_title(f'Inundacao -- JRC South America\n(via climada_petals, ID={RF_SA_ID})')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(left=0)
    ax1.set_ylim(bottom=0)

    mdr_hw = impf_heat.mdd * impf_heat.paa
    ax2.plot(impf_heat.intensity, mdr_hw * 100, 'r-s', linewidth=2, markersize=6)
    ax2.fill_between(impf_heat.intensity, 0, mdr_hw * 100, alpha=0.15, color='red')
    ax2.set_xlabel('Temperatura acima de 40 deg_C')
    ax2.set_ylabel('MDR (%)')
    ax2.set_title(f'Ondas de Calor -- Industrial (Refinaria)\n(customizada, ID={HW_ID})')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(left=0)
    ax2.set_ylim(bottom=0)

    plt.tight_layout()
    plt.savefig('nb03_impact_functions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("  Grafico salvo: nb03_impact_functions.png")
except Exception as e:
    print(f"  Plot nao disponivel: {e}")

In [ ]:
# =============================================================================
# BLOCO 8: CALCULO DE IMPACTO SEPARADO POR HAZARD
# =============================================================================
print("--- Calculando Impacto por Hazard ---")

# 8a. Impacto de inundacao
imp_flood = ImpactCalc(exp, impf_set, haz_flood).impact(save_mat=True)
eai_flood = float(imp_flood.aai_agg)
print(f"\n  INUNDACAO:")
print(f"    EAI: USD {eai_flood:,.0f}")
print(f"    Ratio: {(imp_flood.eai_exp[0]/REDUC_VALUE_USD)*100:.2f}%")
print(f"    Impacto por RP:")
for i, evt in enumerate(events_rf):
    print(f"      RP {evt['rp']:>3}a: USD {imp_flood.at_event[i]:>15,.0f}")

# 8b. Impacto de calor
imp_heat = ImpactCalc(exp, impf_set, haz_heat).impact(save_mat=True)
eai_heat = float(imp_heat.aai_agg)
print(f"\n  ONDAS DE CALOR:")
print(f"    EAI: USD {eai_heat:,.0f}")
print(f"    Ratio: {(imp_heat.eai_exp[0]/REDUC_VALUE_USD)*100:.2f}%")
print(f"    Impacto por RP:")
for i, evt in enumerate(events_hw):
    print(f"      RP {evt['rp']:>3}a: USD {imp_heat.at_event[i]:>15,.0f}")

# 8c. Agregacao
eai_total = eai_flood + eai_heat
eai_ratio_total = (eai_total / REDUC_VALUE_USD) * 100
contrib_rf = (eai_flood / eai_total) * 100
contrib_hw = (eai_heat / eai_total) * 100

print(f"\n  AGREGADO:")
print(f"    EAI Total: USD {eai_total:,.0f}")
print(f"    Ratio Total: {eai_ratio_total:.2f}%")
print(f"    Contribuicao RF: {contrib_rf:.1f}%")
print(f"    Contribuicao HW: {contrib_hw:.1f}%")
print(f"    Metodo: soma simples (independencia assumida)")

In [ ]:
# =============================================================================
# BLOCO 9: PAINEL COMPARATIVO (4 GRAFICOS)
# =============================================================================
print("--- Gerando Painel Comparativo ---")

try:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(
        f'CLIMARISK-OG | Analise Multi-Hazard | {REDUC_NAME}\n'
        f'EAI Total: USD {eai_total/1e6:,.0f}M/ano ({eai_ratio_total:.1f}% do valor do ativo)',
        fontsize=14, fontweight='bold'
    )

    # --- 9a. Pizza: contribuicao por hazard ---
    ax = axes[0, 0]
    sizes = [contrib_rf, contrib_hw]
    labels = [f'Inundacao\nUSD {eai_flood/1e6:,.0f}M\n({contrib_rf:.1f}%)',
              f'Ondas de Calor\nUSD {eai_heat/1e6:,.0f}M\n({contrib_hw:.1f}%)']
    colors = ['#3498db', '#e74c3c']
    ax.pie(sizes, labels=labels, colors=colors, autopct='', startangle=90,
           textprops={'fontsize': 10})
    ax.set_title('Contribuicao por Hazard', fontsize=12, fontweight='bold')

    # --- 9b. Barras: impacto por RP ---
    ax = axes[0, 1]
    rps_rf = [e['rp'] for e in events_rf]
    rps_hw = [e['rp'] for e in events_hw]
    impacts_rf = [imp_flood.at_event[i] / 1e6 for i in range(n_events_rf)]
    impacts_hw = [imp_heat.at_event[i] / 1e6 for i in range(n_events_hw)]

    x_rf = np.arange(len(rps_rf))
    x_hw = np.arange(len(rps_hw)) + len(rps_rf) + 1
    width = 0.7

    bars1 = ax.bar(x_rf, impacts_rf, width, color='#3498db', label='Inundacao')
    bars2 = ax.bar(x_hw, impacts_hw, width, color='#e74c3c', label='Ondas de Calor')

    all_x = np.concatenate([x_rf, x_hw])
    all_labels = [f'RF\nRP{rp}' for rp in rps_rf] + [f'HW\nRP{rp}' for rp in rps_hw]
    ax.set_xticks(all_x)
    ax.set_xticklabels(all_labels, fontsize=8)
    ax.set_ylabel('Impacto (USD milhoes)')
    ax.set_title('Impacto por Periodo de Retorno', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

    # --- 9c. Curvas de excedencia sobrepostas ---
    ax = axes[1, 0]

    freq_rf = np.sort(imp_flood.frequency)[::-1]
    impact_rf_sorted = np.sort(imp_flood.at_event)
    exc_freq_rf = np.cumsum(freq_rf)

    freq_hw = np.sort(imp_heat.frequency)[::-1]
    impact_hw_sorted = np.sort(imp_heat.at_event)
    exc_freq_hw = np.cumsum(freq_hw)

    ax.semilogy(impact_rf_sorted / 1e6, 1.0 / exc_freq_rf, 'b-o',
                linewidth=2, markersize=8, label='Inundacao')
    ax.semilogy(impact_hw_sorted / 1e6, 1.0 / exc_freq_hw, 'r-s',
                linewidth=2, markersize=8, label='Ondas de Calor')
    ax.set_xlabel('Impacto (USD milhoes)')
    ax.set_ylabel('Periodo de Retorno (anos)')
    ax.set_title('Curvas de Excedencia', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3, which='both')

    # --- 9d. Barra horizontal: EAI comparativo ---
    ax = axes[1, 1]
    categories = ['Inundacao', 'Ondas de Calor', 'TOTAL']
    values = [eai_flood / 1e6, eai_heat / 1e6, eai_total / 1e6]
    bar_colors = ['#3498db', '#e74c3c', '#2c3e50']

    bars = ax.barh(categories, values, color=bar_colors, height=0.5)
    ax.set_xlabel('EAI (USD milhoes/ano)')
    ax.set_title('Expected Annual Impact (EAI)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')

    for bar, val in zip(bars, values):
        ax.text(val + max(values)*0.02, bar.get_y() + bar.get_height()/2,
                f'USD {val:,.0f}M', va='center', fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.savefig('nb03_multihazard_panel.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("  OK - Painel salvo: nb03_multihazard_panel.png")
except Exception as e:
    print(f"  Erro no painel: {e}")

In [ ]:
# =============================================================================
# BLOCO 10: RESUMO EXECUTIVO
# =============================================================================
print("\n" + "=" * 70)
print("  RESUMO EXECUTIVO -- CLIMARISK-OG | Notebook 03 | Multi-Hazard")
print("=" * 70)
print(f"""
  ATIVO:       {REDUC_NAME}
  LOCALIZACAO: Duque de Caxias, RJ ({REDUC_LAT}, {REDUC_LON})
  VALOR:       USD {REDUC_VALUE_USD/1e9:.0f} bilhoes
  CENARIO:     Baseline (historico)
  HAZARDS:     Inundacao Fluvial (RF) + Ondas de Calor (HW)

  RESULTADOS PRINCIPAIS:
  ---------------------------------------------------
  EAI Inundacao:       USD {eai_flood:>15,.0f}  ({contrib_rf:.1f}%)
  EAI Ondas de Calor:  USD {eai_heat:>15,.0f}  ({contrib_hw:.1f}%)
  EAI TOTAL:           USD {eai_total:>15,.0f}  (100%)
  Ratio EAI/Valor:     {eai_ratio_total:>14.2f}%
  ---------------------------------------------------

  FUNCOES DE DANO:
    RF: JRC South America (Huizinga et al. 2017) -- via climada_petals
    HW: Custom Industrial (ECA/McKinsey 2009, Kjellstrom 2016, ILO 2019)

  NOTA: Independencia entre hazards assumida (soma simples de EAIs).

  LIMITACOES (TRL5):
    1. Dados de hazard sinteticos para ambos os riscos
    2. Valor de exposicao estimado
    3. Funcao de dano de calor customizada (nao calibrada com dados reais)
    4. Funcao de dano de inundacao generica (residencial)
    5. Independencia entre hazards assumida (sem correlacao)
    6. Cenario baseline apenas
    7. Ativo unico
    8. Soma simples de EAIs (sem efeitos de segunda ordem)

======================================================================
  FIM DO NOTEBOOK 03 -- Versao 1.1
======================================================================
""")

In [ ]:
# =============================================================================
# BLOCO 11: EXPORTACAO JSON
# =============================================================================
import json

results = {
    "metadata": {
        "notebook": "nb03_multihazard_duque_caxias",
        "version": "1.1",
        "date": datetime.now().isoformat(),
        "climada_version": ver,
        "methodology": "CLIMADA multi-hazard H x E x V probabilistic impact",
        "n_hazards": 2
    },
    "asset": {
        "name": REDUC_NAME,
        "lat": REDUC_LAT,
        "lon": REDUC_LON,
        "value_usd": REDUC_VALUE_USD
    },
    "hazards": {
        "RF": {
            "type": "RF",
            "type_name": "River Flood",
            "n_events": n_events_rf,
            "return_periods": [e['rp'] for e in events_rf],
            "intensity_unit": "m",
            "max_intensity": float(haz_flood.intensity.max()),
            "impact_function": {
                "name": impf_flood.name,
                "id": int(impf_flood.id) if isinstance(impf_flood.id, (int, np.integer)) else str(impf_flood.id),
                "source": "Huizinga et al. (2017), doi: 10.2760/16510",
                "loaded_via": "climada_petals.entity.impact_funcs.river_flood.flood_imp_func_set()",
                "type": "structural_damage"
            },
            "results": {
                "eai_usd": eai_flood,
                "eai_ratio_pct": float((imp_flood.eai_exp[0] / REDUC_VALUE_USD) * 100),
                "impact_by_return_period": {
                    str(evt['rp']): float(imp_flood.at_event[i])
                    for i, evt in enumerate(events_rf)
                }
            }
        },
        "HW": {
            "type": "HW",
            "type_name": "Heat Wave",
            "n_events": n_events_hw,
            "return_periods": [e['rp'] for e in events_hw],
            "intensity_unit": "deg_C above threshold",
            "threshold_c": HEAT_THRESHOLD_C,
            "max_intensity": float(haz_heat.intensity.max()),
            "impact_function": {
                "name": "Heat Wave -- Industrial Facility (Refinery)",
                "source": "Custom -- ECA/McKinsey (2009), Kjellstrom et al. (2016), ILO (2019)",
                "type": "operational_loss"
            },
            "results": {
                "eai_usd": eai_heat,
                "eai_ratio_pct": float((imp_heat.eai_exp[0] / REDUC_VALUE_USD) * 100),
                "impact_by_return_period": {
                    str(evt['rp']): float(imp_heat.at_event[i])
                    for i, evt in enumerate(events_hw)
                }
            }
        }
    },
    "aggregated_results": {
        "eai_total_usd": eai_total,
        "eai_total_ratio_pct": eai_ratio_total,
        "contribution_pct": {
            "RF": contrib_rf,
            "HW": contrib_hw
        },
        "aggregation_method": "simple_sum",
        "note": "Independence between hazards assumed. No correlation modeled."
    },
    "limitations": [
        "Dados de hazard sinteticos para ambos os riscos",
        "Valor de exposicao estimado",
        "Funcao de dano de calor customizada (nao calibrada com dados reais)",
        "Funcao de dano de inundacao generica (residencial)",
        "Independencia entre hazards assumida (sem correlacao)",
        "Cenario baseline apenas",
        "Ativo unico",
        "Soma simples de EAIs (sem efeitos de segunda ordem)"
    ]
}

with open('results_nb03_multihazard_reduc.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\nJSON exportado: results_nb03_multihazard_reduc.json")

In [ ]:
# =============================================================================
# BLOCO 12: DIAGNOSTICO E ARTEFATOS
# =============================================================================
import os
from IPython.display import display, Image

print("=" * 60)
print("  PARTE 1: ARTEFATOS GERADOS")
print("=" * 60)

output_files = [
    'nb03_impact_functions.png',
    'nb03_multihazard_panel.png',
    'results_nb03_multihazard_reduc.json',
]

for f in output_files:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024
        print(f"  OK {f} ({size:.1f} KB)")
        if f.endswith('.png'):
            display(Image(filename=f, width=700))
    else:
        print(f"  FALTA {f}")

print("\n" + "=" * 60)
print("  PARTE 2: DIAGNOSTICO DE CONSISTENCIA")
print("=" * 60)

checks = []

# Check 1: EAI total > 0
ok = eai_total > 0
checks.append(('EAI total > 0', ok, f'USD {eai_total:,.0f}'))

# Check 2: Contribuicoes somam ~100%
sum_contrib = contrib_rf + contrib_hw
ok = abs(sum_contrib - 100.0) < 0.01
checks.append(('Contribuicoes somam 100%', ok, f'{sum_contrib:.2f}%'))

# Check 3: EAI de cada hazard >= 0
ok = eai_flood >= 0 and eai_heat >= 0
checks.append(('EAIs individuais >= 0', ok, f'RF={eai_flood:,.0f}, HW={eai_heat:,.0f}'))

# Check 4: Impactos nao excedem valor do ativo
max_impact = max(max(imp_flood.at_event), max(imp_heat.at_event))
ok = max_impact <= REDUC_VALUE_USD * 1.01
checks.append(('Impacto <= valor do ativo', ok, f'max={max_impact:,.0f}'))

# Check 5: Impact function RF carregada via climada_petals
ok = 'JRC' in impf_flood.name or 'South America' in impf_flood.name
checks.append(('RF via climada_petals', ok, impf_flood.name[:50]))

# Check 6: JSON exportado
ok = os.path.exists('results_nb03_multihazard_reduc.json')
checks.append(('JSON exportado', ok, ''))

# Check 7: Mesmos centroids para ambos os hazards
ok = haz_flood.centroids.size == haz_heat.centroids.size
checks.append(('Mesma grade de centroids', ok,
               f'RF={haz_flood.centroids.size}, HW={haz_heat.centroids.size}'))

for name, passed, detail in checks:
    status = 'OK' if passed else 'FALHA'
    print(f"  [{status}] {name}: {detail}")

n_passed = sum(1 for _, p, _ in checks if p)
print(f"\n  Resultado: {n_passed}/{len(checks)} checks passaram")

if n_passed == len(checks):
    print("  Notebook pronto para commit no GitHub.")
else:
    print("  ATENCAO: Verificar checks que falharam antes de commitar.")

# Mostrar conteudo do JSON
print("\nConteudo do JSON:")
with open('results_nb03_multihazard_reduc.json', 'r') as jf:
    print(json.dumps(json.load(jf), indent=2, ensure_ascii=False))